In [1]:
# -*- coding : utf-8 -*-
import numpy as np
import tensorflow as tf
import configparser as cp
import sys
import os
import time
import warnings

# 경고 메시지 무시
warnings.filterwarnings(action='ignore')

# --- 설정 변수 정의 ---
# 테스트 데이터 파일명
datafile = "/src/data/DataSet/SPECData.npy"
if os.path.isfile(datafile) == False:
    print("%s is not file."%datafile)
    exit()
    
# 진동 분석 모델 디렉토리명
modeldir = "/src/hyebin/model/SPEC-02"
if os.path.isdir(modeldir) == False:
    print("%s is not directory."%modeldir)
    exit()

# 테스트 데이터 설정
# 테스트 데이터 중 최초 데이터 위치
base_index = 0
# 테스트할 데이터의 개수
test_count = 3000

# --- 테스트 데이터 로딩  ---
# 데이터 구조
# data[0] : 학습용 데이터(이미지 개수, 48, 64, 3)
# data[1] : 테스트용 데이터(이미지 개수, 48, 64, 3)
# data[2] : 학습용 데이터의 라벨(이미지 개수, 5)
# data[3] : 테스트용 데이터의 라벨 (이미지 개수, 5)
print("load data : ", datafile)
data = np.load(datafile, allow_pickle=True)
print("load complete")

# 테스트용 데이터와 라벨을 사용함
test_x = data[1][base_index:base_index + test_count]
test_y = data[3][base_index:base_index + test_count]

# --- 진동 분석 모델 로딩 및 분석 실행 ---
with tf.Session(graph=tf.Graph()) as sess:
    # 진동 분석 모델 로딩
    tf.saved_model.loader.load(sess, ["ver1"], modeldir)
    
    # 진동 분석 모델의 변수 획득
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("in:0")
    use_dropout = graph.get_tensor_by_name("use_dropout:0")
    out = graph.get_tensor_by_name("out:0")
    
    # 진동 분석 수행
    # 분석 과정이므로, dropout을 사용하지 않음
    start_time = time.time()
    _out = sess.run([out], feed_dict={x:test_x, use_dropout:False})
    end_time = time.time()
    
    # --- 분석 결과 출력 ---
    # 정답 개수
    correct_count = 0
    # 오답 개수
    incorrect_count = 0
    
    print("=====================================")
    print("index\tpredict\tlabel\tcorrect")
    print("-------------------------------------")
    
    for index in range(0, test_count):
        # 예측값
        predict = np.argmax(_out[0][index], axis=0)
        # 실제값
        label = np.argmax(test_y[index], axis=0)
        
        result = "O"
        
        # 정답일 경우, 정답 카운트 1 증가
        # 오답일 경우, 오답 카운트 1 증가 및 출력시 오른쪽에 X 표시
        if predict == label:
            correct_count = correct_count + 1
        else:
            incorrect_count = incorrect_count + 1
            result = "X"
            message = "%d\t%s\t%s\t%s"%(base_index + index, predict, label, result)
            
            # 현재 이미지 분석 결과 출력 - 틀린것만
            print(message)
    
    # 최종 결과 출력
    print("=====================================")
    print("running time : ", end_time - start_time)
    print("correct count : ", correct_count, "\tcorrect percent:", (correct_count/test_count) * 100, "%")
    print("incorrect count : ", incorrect_count)

load data :  /src/data/DataSet/SPECData.npy
load complete
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /src/hyebin/model/SPEC-02/variables/variables
index	predict	label	correct
-------------------------------------
337	4	1	X
665	2	4	X
751	4	0	X
2102	4	2	X
running time :  3.0927159786224365
correct count :  2996 	correct percent: 99.86666666666667 %
incorrect count :  4
